In [196]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import logging

In [6]:
data = pd.read_csv('../data/vacancies_ru.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209567,v_2484959,Сборщик-упаковщик,c_203256,"['Пользователь ПК', 'Умение работать в команде...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,74,4876,114733,5,2,1
2209568,v_205163,Сварщик на полуавтомат,c_158695,"['Желание работать и зарабатывать', 'Высокая э...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,71,5032,65219,5,5,2
2209569,v_639897,Главный инженер / Технический директор,c_209365,"['Контроль исправности оборудования', 'Инженер...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,41,842,121520,5,5,2
2209570,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,"['Предпечатная подготовка', 'Статистический ан...",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,62,2673,162497,5,5,1


In [93]:
url = "https://api.hh.ru/professional_roles"

response = requests.get(url)

if response.status_code == 200:
    data_list = response.json()

In [94]:
data_list['categories'][0]

{'id': '19',
 'name': 'Автомобильный бизнес',
 'roles': [{'id': '4',
   'name': 'Автомойщик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '5',
   'name': 'Автослесарь, автомеханик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '62',
   'name': 'Мастер-приемщик',
   'accept_incomplete_resumes': True,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False},
  {'id': '70',
   'name': 'Менеджер по продажам, менеджер по работе с клиентами',
   'accept_incomplete_resumes': False,
   'is_default': False,
   'select_deprecated': False,
   'search_deprecated': False}]}

In [95]:
prof_roles_codes = []
for i in range(len(data_list['categories'])):
    for j in range(len(data_list['categories'][i]['roles'])):
          prof_roles_codes.append(int(data_list['categories'][i]['roles'][j]['id']))   

In [279]:
filename = 'vacancy_dataset.csv'
page_cnt = 20
item_per_page = 20
area_code = 113
currency_code = 'RUR'
order_by = 'publication_time'
search_period = 30

url_hh = 'https://api.hh.ru/'
url_roles = url_hh + 'professional_roles'
url_vacancies = url_hh + 'vacancies' 

In [280]:
response = requests.get(url_roles)

if response.status_code != 200:
    print('Error:', response.text)
else:
    data_list = response.json()

    prof_roles_codes = []
    for i in range(len(data_list['categories'])):
        for j in range(len(data_list['categories'][i]['roles'])):
            prof_roles_codes.append(int(data_list['categories'][i]['roles'][j]['id']))

In [281]:
len(prof_roles_codes)

270

In [282]:
columns = ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from',
           'compensation_to', 'compensation_currencyCode', 'area_id', 'employment',
           'workSchedule', 'workExperience', 'description', 'published_at']

In [286]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

In [287]:
logging.basicConfig(
    filename='hh_logs.log',
    level=logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(message)s'
    )

In [ ]:
df = pd.DataFrame(columns=columns)
ind = 0

for code in prof_roles_codes:
    ind += 1
    vacancies = []
    progress_bar = tqdm(total=page_cnt, desc=f"Professional role {code}, {ind}/{len(prof_roles_codes)}")
    for i in range(page_cnt):
        progress_bar.update(1)
        
        params = {'area': area_code,'professional_role': code, 'currency_code': currency_code, 
                  'per_page': item_per_page, 'page': i, 'search_period': search_period, 'order_by': order_by}
        
        response = requests.get(url_vacancies, params=params)
        
        if response.status_code == 200:
            vac_page = response.json()
            
            for j in range(len(vac_page['items'])):
                vac = vac_page['items'][j]
                row = {column: None for column in columns}

                row['vacancy_id'] = vac.get('id')

                response = requests.get(f"{url_vacancies}/{row['vacancy_id']}")

                if response.status_code == 200:
                    vac = response.json()

                    row['name'] = vac.get('name')
                    row['company_id'] = vac['employer'].get('id')
                    row['keySkills'] = vac.get('id')

                    if vac.get('salary') is not None:
                        row['compensation_from'] = vac['salary'].get('from')
                        row['compensation_to'] = vac['salary'].get('to')
                        row['compensation_currencyCode'] = vac['salary'].get('currency')
                    else:
                        row['compensation_from'] = None
                        row['compensation_to'] = None
                        row['compensation_currencyCode'] = None

                    row['area_id'] = vac['area'].get('id')
                    row['employment'] = vac.get('employment').get('id') if vac.get('employment') else None
                    row['workSchedule'] = vac.get('schedule').get('id') if vac.get('schedule') else None
                    row['workExperience'] = vac.get('experience').get('id') if vac.get('experience') else None
                    row['description'] = BeautifulSoup(vac['description'], 'html.parser').get_text()
                    row['keySkills'] = [item['name'] for item in vac.get('key_skills', [])]
                    row['published_at'] = datetime.strptime(vac.get('published_at')[:19],
                                                                  '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d %H:%M:%S')
                    row['alternate_url'] = vac.get('alternate_url')
                    
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                    logging.info(f'Successful concatenation. Shape of dataframe: {df.shape}')

    progress_bar.close()



Professional role 93, 11/270:  55%|██████▌     | 11/20 [35:55<29:23, 195.97s/it]

Professional role 21, 41/270:   0%|                      | 0/20 [00:00<?, ?it/s]

In [ ]:
with open('hh_logs.log', 'r') as file:
    logs = file.readlines()

print(logs[0].strip())

In [266]:
df

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,employment,workSchedule,workExperience,description,published_at,alternate_url
0,96624134,Специалист по предпродажной подготовке автомоб...,610391,[],70000,100000,RUR,2,full,shift,between1And3,"В связи с активным развитием отдела продаж, в ...",2024-05-04 14:28:47,https://hh.ru/vacancy/96624134
1,96623106,Автомаляр,974235,[],100000,200000,RUR,1,full,fullDay,between1And3,На постоянную работу требуется автомаляр с опы...,2024-05-04 14:18:03,https://hh.ru/vacancy/96623106
2,96622101,Мастер по уходу за автомобилями,1317452,[],50000,150000,RUR,26,full,fullDay,noExperience,Приглашаем стать частью нашей команды професси...,2024-05-04 14:08:25,https://hh.ru/vacancy/96622101
3,96349109,Автомойщик,10242739,"[Работать, Зарабатывать, Трудолюбие, Активность]",60000,80000,RUR,66,full,shift,noExperience,Требуются мойщики на премиум автомоечный компл...,2024-05-04 13:56:49,https://hh.ru/vacancy/96349109
4,97122138,Автомойщик,3191302,[желание работать и зарабатывать],45000,50000,RUR,54,full,shift,noExperience,Обязанности: мойка автомобилей для сервиса и ...,2024-05-04 13:51:36,https://hh.ru/vacancy/97122138
5,97121698,Автожестянщик,3061760,[],100000,None,RUR,2039,full,shift,between1And3,Обязанности: Рихтовка металла Подготовка авто...,2024-05-04 13:50:07,https://hh.ru/vacancy/97121698
6,97430971,Битумщик (Козелки),4114984,"[Работа в режиме многозадачности, Работа в ком...",60000,None,RUR,78,full,shift,noExperience,"На производство асфальтобетона в ООО ""АБЗ-1"" т...",2024-05-04 13:47:04,https://hh.ru/vacancy/97430971
7,97121104,Арматурщик,3061760,"[Знание технического устройства автомобиля, Ар...",80000,None,RUR,2039,full,shift,between1And3,Обязанности: Разборка и сборка автомобиля дл...,2024-05-04 13:43:33,https://hh.ru/vacancy/97121104
8,97624480,Мойщик автомобилей,852424,"[Коммуникабельность, ответственность]",30000,None,RUR,212,full,shift,between1And3,Обязанности: Коммерческая и технологическая мо...,2024-05-04 13:42:44,https://hh.ru/vacancy/97624480
9,98388096,Автомаляр-подготовщик / Подготовщик-арматурщик,1683762,[],40000,80000,RUR,72,full,fullDay,between3And6,"Обязанности: Подготовка автомобиля к окрасу, п...",2024-05-04 13:40:37,https://hh.ru/vacancy/98388096


In [192]:
df_new = df.dropna(subset=['name'])
df_new.to_csv(filename, index=False, mode='a')

In [193]:
new_data = pd.read_csv(filename)
new_data.shape

(19516, 13)

In [ ]:
# для апдейта можно search_period поменьше брать

In [219]:
# draft to check fields
response = requests.get(f"{url_vacancies}/{row['vacancy_id']}")

if response.status_code == 200:
    vac = response.json()
    print(vac['alternate_url'])

https://hh.ru/vacancy/92767727
